# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [12]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [1]:
##########################################################################
###                                                                    ###    
### Credit where it's due:                                             ###
### Almost all this work was from Shahram. Running it took hours and I ###
### am including it for reference. I will be referring to a dataset    ###
### That I extracted going forward.                                    ###
###                                                                    ### 
### DO NOT RUN! TAKES FOREVER! MAY GET KICKED OFF INDEED.COM!          ###  
###                                                                    ### 
##########################################################################

import requests
from bs4 import BeautifulSoup
import pandas as pd

INDEED_URL_1='http://www.indeed.com/jobs?q=Data+Scientist&l='
INDEED_URL='http://www.indeed.com/jobs?q=Data+Scientist&start='
project4=pd.DataFrame(columns=['title','company','location','salary','description'])
for i in range (1,100):
   if i==0:
       print i
       INDEED_URL=INDEED_URL_1
       r=requests.get(INDEED_URL)
       soup=BeautifulSoup(r.text,"html")
       results=soup.findAll('div',{"class":" row result"})+soup.findAll('div',{"class":"row result"})
       for result in results:
           title=''
           company=''
           location=''
           salary=''
           description=''
           try:
               temp=result.select('h2.jobtitle')
               title=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('span.company')
               company=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('span.location')
               location=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('nobr')
               salary=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('span.summary')
               description=temp[0].text.strip()
           except:
               pass
           project4.loc[len(project4)]=[title,company,location,salary,description]            

   else:
#         if (i%100==0):
       print i
       INDEED_URL_UNIQUE=INDEED_URL+str(i*10)
       r=requests.get(INDEED_URL_UNIQUE)
       soup=BeautifulSoup(r.text,"html")
       results=soup.findAll('div',{"class":" row result"})+soup.findAll('div',{"class":"row result"})
       for result in results:
           title=''
           company=''
           location=''
           salary=''
           description=''
           try:
               temp=result.select('h2.jobtitle')
               title=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('span.company')
               company=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('span.location')
               location=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('nobr')
               salary=temp[0].text.strip()
           except:
               pass
           try:
               temp=result.select('span.summary')
               description=temp[0].text.strip()
           except:
               pass
           project4.loc[len(project4)]=[title,company,location,salary,description]   
project4

1


/home/mg/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


KeyboardInterrupt: 

In [20]:
##########################################################################
###                                                                    ###    
### Read in file we already have of scraped info                       ###
###                                                                    ### 
##########################################################################
import pandas as pd

project4 = pd.read_csv('../assets/indeed_scrape_proj4.csv')
project4.head()


,Unnamed: 0,title,company,location,salary,description
0,0,"Data Scientist II - Charlotte, NC",Bank of America,"Charlotte, NC",NaN,As well as deployment of advanced techniques (...
1,1,Data Scientist,Simple Finance,"Portland, OR",NaN,"Predictive analytics, experimental design, dat..."
2,2,Associate Data Scientist,PayPal,"San Jose, CA",NaN,Collaborate with other research scientists and...
3,3,Data Scientist,Shuchi Rana Sandbox,"Menlo Park, CA",NaN,Experiment engineers will be expected to help ...
4,4,"Data Scientist, Google Analytics 360, Google T...",Google,"Mountain View, CA",NaN,Lead analytics aspects of client engagements i...


In [1]:
## YOUR CODE HERE

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [2]:
## YOUR CODE HERE

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [5]:
YOUR_CITY = ''

In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        pass

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [2]:
def Number_Splitter(number):
   number_str=list(str(number))
   number_1=''
   number_2=''
   i=len(number_str)/2
   for j in range(0,i):
       number_1+=number_str[j]
   for j in range(i,len(number_str)):
       number_2+=number_str[j]                
   num_out=0.5*(int(number_1)+int(number_2))
   return num_out

X = project4['salary']

for i in range(0,len(X)):
   if X[i]>10000000:
       X[i]=Number_Splitter(X[i])
   # Fix the Hourly (Single)
   if X[i]<300:
       X[i]=2000*X[i]
   # Fix the Month (Single)
   if (X[i]<40000 and X[i]>1000):
       X[i]=12*X[i]
       
"""
temp_cat=[]      
for i in range(0,len(X)):
   t='nan'
   if (X[i]<100000):
       t=0
   if (X[i]>=100000 and X[i]<150000): 
       t=1
   if (X[i]>=150000): 
       t=2
   temp_cat.append(t)
       
test['salary_cat'] = temp_cat
"""

x.head()

ValueError: invalid literal for int() with base 10: '$120,00'

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [14]:
## YOUR CODE HERE

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [12]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [15]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

Unknown without knowing more about the data distribution. 

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [17]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [47]:
# Create dummy variable for 'senior'
-

project4.head(50)

,Unnamed: 0,title,company,location,salary,description,senior,manager,expert,machlearn,bigdata
0,0,"Data Scientist II - Charlotte, NC",Bank of America,"Charlotte, NC",NaN,As well as deployment of advanced techniques (...,False,False,False,True,False
1,1,Data Scientist,Simple Finance,"Portland, OR",NaN,"Predictive analytics, experimental design, dat...",False,False,False,True,False
2,2,Associate Data Scientist,PayPal,"San Jose, CA",NaN,Collaborate with other research scientists and...,False,False,False,False,False
3,3,Data Scientist,Shuchi Rana Sandbox,"Menlo Park, CA",NaN,Experiment engineers will be expected to help ...,False,False,False,False,False
4,4,"Data Scientist, Google Analytics 360, Google T...",Google,"Mountain View, CA",NaN,Lead analytics aspects of client engagements i...,False,False,False,False,False
5,5,USA - Data Scientist,Appnomic Systems Pvt. Ltd,"Sunnyvale, CA",NaN,We’re looking for a data scientist to help us ...,False,False,False,False,False
6,6,Data Scientist,Simplion Technologies Inc,"Sunnyvale, CA",NaN,Experienced in developing machine learning mod...,False,False,False,True,False
7,7,Data Scientist,Lawrence Livermore National Laboratory,"Livermore, CA 94550",NaN,"Data intensive applications, text processing, ...",False,False,False,True,False
8,8,Data Scientist,SPIN Analytics and Strategy LLC,"San Jose, CA",NaN,Communicate insights and recommend areas for f...,False,False,False,False,False
9,9,NaN,Uber,"New York, NY",NaN,Technical understanding must go from the highe...,False,False,False,False,False


#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [19]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [20]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [21]:
## YOUR CODE HERE

In [22]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [23]:
## YOUR CODE HERE

In [24]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [25]:
## YOUR CODE HERE